In [1]:
import os
import time
os.environ["CUDA_LAUNCH_BLOCKING"] = "1"
from os import listdir
import pandas as pd
import numpy as np
import glob
import cv2
import json
from os.path import expanduser
import splitfolders
import shutil
from define_path import Def_Path

from tqdm import tqdm

import torch 
import torchvision
from torchvision import models
from torchvision.models.detection.rpn import AnchorGenerator
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn 
import torchvision.transforms as T
from torchvision.transforms import functional as F
from torchsummary import summary

from sklearn.model_selection import train_test_split

import albumentations as A # Library for augmentations

import matplotlib.pyplot as plt 
from PIL import Image

import transforms, utils, engine, train
from utils import collate_fn
from engine import train_one_epoch, evaluate

t = torch.cuda.get_device_properties(0).total_memory
print(t)
torch.cuda.empty_cache()

r = torch.cuda.memory_reserved(0)
print(r)
a = torch.cuda.memory_allocated(0)
print(a)
# f = r-a  # free inside reserved

weights_path = '/home/jc-merlab/Pictures/Data/trained_models/keypointsrcnn_weights_sim_b1_e25_v0.pth'

16908615680
0
0


In [2]:
# to generalize home directory. User can change their parent path without entering their home directory
path = Def_Path()

parent_path =  path.home + "/Pictures/" + "Data/"

root_dir = parent_path + path.year + "-" + path.month + "-" + path.day + "/"

In [3]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
# torch.cuda.set_per_process_memory_fraction(0.9, 0)
print(device)

cuda


In [4]:
def train_transform():
    return A.Compose([
        A.Sequential([
            A.RandomRotate90(p=1), # Random rotation of an image by 90 degrees zero or more times
            A.RandomBrightnessContrast(brightness_limit=0.2, contrast_limit=0.3, brightness_by_max=True, always_apply=False, p=1), # Random change of brightness & contrast
        ], p=1)
#         A.Resize(640, 480)  # Resize all images to be 640x480
    ],
    keypoint_params=A.KeypointParams(format='xy'), # More about keypoint formats used in albumentations library read at https://albumentations.ai/docs/getting_started/keypoints_augmentation/
    bbox_params=A.BboxParams(format='pascal_voc', label_fields=['bboxes_labels']) # Bboxes should have labels, read more at https://albumentations.ai/docs/getting_started/bounding_boxes_augmentation/
    )

In [5]:
def train_test_split(src_dir):
    dst_dir_img = src_dir + "images"
    dst_dir_anno = src_dir + "annotations"
    
    if os.path.exists(dst_dir_img) and os.path.exists(dst_dir_anno):
        print("folders exist")
    else:
        os.mkdir(dst_dir_img)
        os.mkdir(dst_dir_anno)
        
    for jpgfile in glob.iglob(os.path.join(src_dir, "*.jpg")):
        shutil.copy(jpgfile, dst_dir_img)

    for jsonfile in glob.iglob(os.path.join(src_dir, "*.json")):
        shutil.copy(jsonfile, dst_dir_anno)
        
    output = parent_path + "split_folder_output" + "-" + path.year + "-" + path.month + "-" + path.day 
    
    splitfolders.ratio(src_dir, # The location of dataset
                   output=output, # The output location
                   seed=42, # The number of seed
                   ratio=(.7, .2, .1), # The ratio of split dataset
                   group_prefix=None, # If your dataset contains more than one file like ".jpg", ".pdf", etc
                   move=False # If you choose to move, turn this into True
                   )
    
    shutil.rmtree(dst_dir_img)
    shutil.rmtree(dst_dir_anno)
    
    return output  

In [6]:
class KPDataset(Dataset):
    def __init__(self, root, transform=None, demo=False):                
        self.root = root
        self.transform = transform
        self.demo = demo # Use demo=True if you need transformed and original images (for example, for visualization purposes)
        self.imgs_files = sorted(os.listdir(os.path.join(root, "images")))
        self.annotations_files = sorted(os.listdir(os.path.join(root, "annotations")))
    
    def __getitem__(self, idx):
        img_file = self.imgs_files[idx]
        img_path = os.path.join(self.root, "images", self.imgs_files[idx])
        annotations_path = os.path.join(self.root, "annotations", self.annotations_files[idx])

        img_original = cv2.imread(img_path)
        img_original = cv2.cvtColor(img_original, cv2.COLOR_BGR2RGB)
        
        with open(annotations_path) as f:
            data = json.load(f)
            bboxes_original = data['bboxes']
            keypoints_original = data['keypoints']
            
            # All objects are keypoints on the robot
            bboxes_labels_original = [] 
            bboxes_labels_original.append('base_joint')
            bboxes_labels_original.append('joint2')
            bboxes_labels_original.append('joint3')
            bboxes_labels_original.append('joint4')
            bboxes_labels_original.append('joint5')
            bboxes_labels_original.append('joint6')  

        if self.transform:   
            # Converting keypoints from [x,y,visibility]-format to [x, y]-format + Flattening nested list of keypoints            
            # For example, if we have the following list of keypoints for three objects (each object has two keypoints):
            # [[obj1_kp1, obj1_kp2], [obj2_kp1, obj2_kp2], [obj3_kp1, obj3_kp2]], where each keypoint is in [x, y]-format            
            # Then we need to convert it to the following list:
            # [obj1_kp1, obj1_kp2, obj2_kp1, obj2_kp2, obj3_kp1, obj3_kp2]
            keypoints_original_flattened = [el[0:2] for kp in keypoints_original for el in kp]
            
            # Apply augmentations
            transformed = self.transform(image=img_original, bboxes=bboxes_original, bboxes_labels=bboxes_labels_original, keypoints=keypoints_original_flattened)
            img = transformed['image']
            bboxes = transformed['bboxes']
            # Unflattening list transformed['keypoints']
            # For example, if we have the following list of keypoints for three objects (each object has two keypoints):
            # [obj1_kp1, obj1_kp2, obj2_kp1, obj2_kp2, obj3_kp1, obj3_kp2], where each keypoint is in [x, y]-format
            # Then we need to convert it to the following list:
            # [[obj1_kp1, obj1_kp2], [obj2_kp1, obj2_kp2], [obj3_kp1, obj3_kp2]]
            keypoints_transformed_unflattened = np.reshape(np.array(transformed['keypoints']), (-1,1,2)).tolist()

            # Converting transformed keypoints from [x, y]-format to [x,y,visibility]-format by appending original visibilities to transformed coordinates of keypoints
            keypoints = []
            for o_idx, obj in enumerate(keypoints_transformed_unflattened):
#                 print("object", obj)
#                 print(" obj index", o_idx)# Iterating over objects
                obj_keypoints = []
                for k_idx, kp in enumerate(obj): # Iterating over keypoints in each object
                    obj_keypoints.append(kp + [keypoints_original[o_idx][k_idx][2]])
                keypoints.append(obj_keypoints)
        
        else:
            img, bboxes, keypoints = img_original, bboxes_original, keypoints_original        
        
        # Convert everything into a torch tensor        
        bboxes = torch.as_tensor(bboxes, dtype=torch.float32)       
        target = {}
        labels = [1, 2, 3, 4, 5, 6]            
        target["boxes"] = bboxes
        target["labels"] = torch.as_tensor(labels, dtype=torch.int64) # all objects are joint positions
        target["image_id"] = torch.tensor([idx])
        target["area"] = (bboxes[:, 3] - bboxes[:, 1]) * (bboxes[:, 2] - bboxes[:, 0])
        target["iscrowd"] = torch.zeros(len(bboxes), dtype=torch.int64)
        target["keypoints"] = torch.as_tensor(keypoints, dtype=torch.float32)
        img = F.to_tensor(img)        
        bboxes_original = torch.as_tensor(bboxes_original, dtype=torch.float32)
        target_original = {}
        target_original["boxes"] = bboxes_original
        target_original["labels"] = torch.as_tensor(labels, dtype=torch.int64) # all objects are glue tubes
        target_original["image_id"] = torch.tensor([idx])
        target_original["area"] = (bboxes_original[:, 3] - bboxes_original[:, 1]) * (bboxes_original[:, 2] - bboxes_original[:, 0])
        target_original["iscrowd"] = torch.zeros(len(bboxes_original), dtype=torch.int64)
        target_original["keypoints"] = torch.as_tensor(keypoints_original, dtype=torch.float32)        
        img_original = F.to_tensor(img_original)

        if self.demo:
            return img, target, img_original, target_original, img_file
        else:
            return img, target, img_file
    
    def __len__(self):
        return len(self.imgs_files)

In [7]:
# import torch
# import torch.nn.functional as functional
# from torch.nn import Sequential as Seq, Linear, ReLU
# from torch_geometric.nn import GATConv
# from torch_geometric.data import Data

# class GAT(torch.nn.Module):
#     def __init__(self):
#         super(GAT, self).__init__()
#         self.conv1 = GATConv(5, 16, heads=4, dropout=0.4)
#         self.conv2 = GATConv(64, 16, heads=4, dropout=0.4)
#         self.conv3 = GATConv(64, 16, heads=4, dropout=0.4)  # New layer
#         self.conv4 = GATConv(64, 32, heads=4, dropout=0.4)  # New layer

#     def forward(self, x, edge_index):
#         x = functional.elu(self.conv1(x, edge_index))
#         x = functional.elu(self.conv2(x, edge_index))
#         x = functional.elu(self.conv3(x, edge_index))  # Use the new layer
#         x = self.conv4(x, edge_index)  # Use the new layer
#         return x  # This is the predicted keypoints. Shape should be [num_nodes, num_node_features]

In [8]:
import torch
import torch.nn.functional as func
from torch_geometric.nn import GCNConv
from torch_geometric.data import Data

class KeyPointGNN(torch.nn.Module):
    def __init__(self):
        super(KeyPointGNN, self).__init__()
        # Define graph convolution layers
        self.conv1 = GCNConv(2, 256) # 5 features for each keypoint
        self.conv2 = GCNConv(256, 128)
        self.conv3 = GCNConv(128,64)
        self.fc = torch.nn.Linear(64, 2) # Output 5 values (x, y, c, v, label) for each keypoint
        
        # Linear layers for edge feature prediction
        self.edge_fc1 = torch.nn.Linear(2, 128) # Assuming edge_features has 2 elements
        self.edge_fc2 = torch.nn.Linear(128, 2) # Output size depends on your edge features

    def forward(self, data):
        x, edge_index = data.x, data.edge_index
        # Graph convolution layers
        x = func.relu(self.conv1(x, edge_index))
        x = func.relu(self.conv2(x, edge_index))
        x = func.relu(self.conv3(x, edge_index))
        # Fully connected layer to output keypoint locations
        x = self.fc(x)
        
        # Linear layers for edge feature prediction
        edge_features = func.relu(self.edge_fc1(edge_features))
        predicted_edge_features = self.edge_fc2(edge_features)
        
        return x, predicted_edge_features

In [9]:
class KeypointPipeline(nn.Module):
    def __init__(self, weights_path, num_expected_keypoints, delta=1.0):
        super().__init__()

        self.keypoint_rcnn = torchvision.models.detection.keypointrcnn_resnet50_fpn(pretrained=False, pretrained_backbone=True, num_keypoints=num_expected_keypoints, num_classes=7)
        self.num_expected_keypoints = num_expected_keypoints
        self.delta = delta

        # Integrate the GAT model
        self.gcn = KeyPointGNN().to(device)


    def get_edge_features(self, keypoints):
        edges = [[0, 1, 2, 3, 4, 5], [1, 2, 3, 4, 5, 0]]
        return torch.tensor(edges, dtype=torch.long)
    
    def complete_missing_keypoints(self, keypoints, labels, num_expected_keypoints=6):

        detected_kps = keypoints.shape[0]
        # Check if all keypoints are detected
        if detected_kps == num_expected_keypoints:
            return keypoints
        
        # Create a placeholder tensor for keypoints with the correct shape
        ordered_keypoints = torch.zeros((num_expected_keypoints, 3), device=keypoints.device)

        # If some keypoints are detected, compute their average position
        average_kp = torch.mean(keypoints, dim=0, keepdim=True)

        for i, label in enumerate(labels):
            ordered_keypoints[label - 1] = keypoints[i]

        # Fill in the missing keypoints with the average position
        missing_indices = (torch.sum(ordered_keypoints, dim=1) == 0).nonzero(as_tuple=True)[0]
        ordered_keypoints[missing_indices] = average_kp

        return ordered_keypoints
    
    def forward(self, images, adj_matrix=None, targets=None, train=False):
        if train:
            output = self.keypoint_rcnn(images, targets)
            return output
    
        else:
            all_keypoints = []
            with torch.no_grad():
                self.keypoint_rcnn.eval()
                output = self.keypoint_rcnn(images)
                self.keypoint_rcnn.train()

                for i in range(len(images)):  # Process each image in the batch
                    print("image in sample", i)
                    keypoints = output[i]['keypoints'].detach().cpu().numpy()
                    kp_score = output[i]['keypoints_scores'].detach().cpu().numpy()
                    labels = output[i]['labels'].detach().cpu().numpy()
                    unique_labels = list(set(labels))
                    scores = output[i]['scores'].detach().cpu().numpy()
                    print("labels", unique_labels)

                    kps = []
                    kp_scores = []
                    ulabels = []

                    for label in unique_labels:
                        indices = [j for j, x in enumerate(labels) if x == label]
                        scores_for_label = [scores[j] for j in indices]
                        max_score_index = indices[scores_for_label.index(max(scores_for_label))]
                        kp_score_label = kp_score[max_score_index].tolist()
                        kps.append(keypoints[max_score_index][kp_score_label.index(max(kp_score_label))])
                        ulabels.append(label)

                    kps = [torch.tensor(kp, dtype=torch.float32) for kp in kps]
                    if not kps:
                        default_value = torch.tensor([[320, 240, 1]], dtype=torch.float32, device=images[i].device)
                        keypoints = default_value.repeat(6, 1)
                    else:
                        keypoints = torch.stack(kps)
                        
                    print("kp before placeholder", keypoints)
                    keypoints = self.complete_missing_keypoints(keypoints, unique_labels)[:, 0:2]
                    print("kp after placeholder", keypoints)
                    edges = self.get_edges()
                    data = Data(x=keypoints, edge_index=edges)
                    updated_keypoints = self.gcn(data.x, data.edge_index)
#                     keypoints = nn.functional.relu(keypoints)
#                     keypoints = nn.functional.relu(keypoints)
                    print("kp after graph", updated_keypoints)
#                     print("denormalized kp after graph", denormalize_keypoints(keypoints))
#                     keypoints = self.graph_conv3(keypoints, adj_matrix)
                    all_keypoints.append(updated_keypoints)
                    all_edges.append(updated_keypoints)
            print("All keypoints", all_keypoints)

            return torch.stack(all_keypoints), 


#         return pos_loss

In [ ]:
def custom_loss(self, vertices_pred, vertices_orig, pred_edge_features, orig_edge_features):
        # Assume both tensors are of the shape [num_nodes, num_node_features] and the first two features are x and y.
#         pos_loss = functional.mse_loss(pred[:, :3], orig[:, :3])  # Loss based on position of keypoints
        # Compute differences
        diff = (vertices_pred[:, :3] - vertices_orig[:, :3]).abs()

        # Compute Huber loss
        huber_loss = torch.where(diff < self.delta, 0.5 * diff**2, self.delta * (diff - 0.5 * self.delta))
        pos_loss = huber_loss.mean()
        # Compute loss for edge features
        edge_loss = func.mse_loss(pred_edge_features, orig_edge_features)

        vis_loss = func.cross_entropy(vertices_pred[:, 3], vertices_orig[:, 3])  # Loss based on visibility of keypoints

        return pos_loss + vis_loss + edge_loss

In [10]:
# import torch.optim as optim

# num_epochs = 1  # Define your number of epochs
# batch_size = 1

# KEYPOINTS_FOLDER_TRAIN = train_test_split(root_dir) +"/train" #train_test_split(root_dir) +"/train"
# KEYPOINTS_FOLDER_VAL = train_test_split(root_dir) +"/val"
# KEYPOINTS_FOLDER_TEST = train_test_split(root_dir) +"/test"

# dataset_train = KPDataset(KEYPOINTS_FOLDER_TRAIN, transform=None, demo=False)
# dataset_val = KPDataset(KEYPOINTS_FOLDER_VAL, transform=None, demo=False)
# dataset_test = KPDataset(KEYPOINTS_FOLDER_TEST, transform=None, demo=False)

# data_loader_train = DataLoader(dataset_train, batch_size=batch_size, shuffle=True, collate_fn=collate_fn)
# data_loader_val = DataLoader(dataset_val, batch_size=1, shuffle=False, collate_fn=collate_fn)
# data_loader_test = DataLoader(dataset_test, batch_size=1, shuffle=False, collate_fn=collate_fn)


# model = KeypointPipeline(weights_path, num_vertices=6)  # Initialize the model with the provided weights and vertices
# model = model.to(device)  # Ensure the model is on the right device

# optimizer = optim.Adam(model.parameters(), lr=0.005)  # Define the optimizer

# v=2

# for epoch in range(num_epochs):
#     start_time = time.time()
#     model.train()  # Ensure the model is in training mode    

#     # Training loop
#     for i, batch in enumerate(data_loader_train):
#         img_tuple, target_dict_tuple, img_files = batch
        
#         imgs = [img.to(device) for img in img_tuple]  # Create list of images
        
#         # Process each image individually
#         total_train_loss = 0
#         for i in range(len(imgs)):
#             img = imgs[i].unsqueeze(0)  # Unsqueeze to add batch dimension
            
#             # Prepare ground truth vertices for the image
#             keypoints = target_dict_tuple[i]['keypoints'].to(device)
#             visibility = torch.ones((keypoints.shape[0], keypoints.shape[1], 1)).to(device)
#             vertices_gt = torch.cat((keypoints, visibility), dim=2).unsqueeze(0)  # Unsqueeze to add batch dimension
            
#             vertices_gt = vertices_gt.squeeze()
            
#             print("ground truth keypoints", vertices_gt.shape)
                        
#             optimizer.zero_grad()  # Clear the gradients

#             outputs = model(img)  # Forward pass
            
#             vertices_pred, edge_pred, edge_features = outputs[0]

#             loss = model.loss_function(vertices_pred, vertices_gt, edge_pred, edge_features)  # Compute the loss
#             loss.backward()  # Backpropagate the loss

#             optimizer.step()  # Update the model parameters

#             total_train_loss += loss.item()

#     end_time = time.time()
#     epoch_time = end_time - start_time
#     eta = epoch_time * (num_epochs - epoch - 1)
#     print(f'Epoch {epoch+1}/{num_epochs}, Loss: {loss.item()}, ETA: {eta} seconds')
    

#     model.eval()  # Ensure the model is in evaluation mode
    

#     # Validation loop
#     with torch.no_grad():  # No need to track gradients in validation
#         for i, batch in enumerate(data_loader_val):
#             img_tuple, target_dict_tuple, img_files = batch
#             imgs = [img.to(device) for img in img_tuple]  # Create list of images          

            
#             total_val_loss = 0
#             for i in range(len(imgs)):
#                 img = imgs[i].unsqueeze(0)  # Unsqueeze to add batch dimension
#                 # Prepare ground truth vertices for the image
#                 keypoints = target_dict_tuple[i]['keypoints'].to(device)
#                 visibility = torch.ones((keypoints.shape[0], keypoints.shape[1], 1)).to(device)
#                 vertices_gt = torch.cat((keypoints, visibility), dim=2).unsqueeze(0)  # Unsqueeze to add batch dimension

#                 vertices_gt = vertices_gt.squeeze()
                
#                 outputs = model(img)  # Forward pass
            
#                 vertices_pred, edge_pred, edge_features = outputs[0]

#                 loss = model.loss_function(vertices_pred, vertices_gt, edge_pred, edge_features)  # Compute the loss

#                 total_val_loss += loss.item()

#         print(f'Validation Loss after epoch {epoch + 1}: {total_val_loss/len(data_loader_val)}')
    
# model_save_path = f"/home/jc-merlab/Pictures/Data/trained_models/keypointsrcnn_weights_gcn_b{batch_size}_e{num_epochs}_v{v}.pth"
# torch.save(model, model_save_path)

Copying files: 2662 files [00:00, 17794.14 files/s]
Copying files: 2662 files [00:00, 17141.24 files/s]
Copying files: 2662 files [00:00, 17565.84 files/s]


ground truth keypoints torch.Size([6, 4])
vertices for graph node features tensor([[258.0000, 367.0000,   1.0000,   1.0000,   1.0000],
        [258.0000, 283.0000,   0.9999,   1.0000,   2.0000],
        [232.0000, 207.0000,   0.9998,   1.0000,   3.0000],
        [251.0000, 201.0000,   0.9996,   1.0000,   4.0000],
        [242.0000, 102.0000,   0.9993,   1.0000,   5.0000],
        [228.0000,  83.0000,   0.9989,   1.0000,   6.0000]], device='cuda:0')
Vertices in Node features tensor([[258.0000, 367.0000,   1.0000,   1.0000,   1.0000],
        [258.0000, 283.0000,   0.9999,   1.0000,   2.0000],
        [232.0000, 207.0000,   0.9998,   1.0000,   3.0000],
        [251.0000, 201.0000,   0.9996,   1.0000,   4.0000],
        [242.0000, 102.0000,   0.9993,   1.0000,   5.0000],
        [228.0000,  83.0000,   0.9989,   1.0000,   6.0000]], device='cuda:0')
ground truth keypoints torch.Size([6, 4])
vertices for graph node features tensor([[258.0000, 367.0000,   1.0000,   1.0000,   1.0000],
        

vertices for graph node features tensor([[258.0000, 367.0000,   1.0000,   1.0000,   1.0000],
        [258.0000, 283.0000,   0.9999,   1.0000,   2.0000],
        [281.0000, 207.0000,   0.9998,   1.0000,   3.0000],
        [301.0000, 213.0000,   0.9999,   1.0000,   4.0000],
        [323.0000, 308.0000,   0.9967,   1.0000,   5.0000],
        [339.0000, 297.0000,   0.9970,   1.0000,   6.0000]], device='cuda:0')
Vertices in Node features tensor([[258.0000, 367.0000,   1.0000,   1.0000,   1.0000],
        [258.0000, 283.0000,   0.9999,   1.0000,   2.0000],
        [281.0000, 207.0000,   0.9998,   1.0000,   3.0000],
        [301.0000, 213.0000,   0.9999,   1.0000,   4.0000],
        [323.0000, 308.0000,   0.9967,   1.0000,   5.0000],
        [339.0000, 297.0000,   0.9970,   1.0000,   6.0000]], device='cuda:0')
ground truth keypoints torch.Size([6, 4])
vertices for graph node features tensor([[258.0000, 367.0000,   1.0000,   1.0000,   1.0000],
        [258.0000, 283.0000,   0.9999,   1.0000,  

vertices for graph node features tensor([[258.0000, 367.0000,   1.0000,   1.0000,   1.0000],
        [258.0000, 283.0000,   0.9999,   1.0000,   2.0000],
        [233.0000, 207.0000,   0.9997,   1.0000,   3.0000],
        [252.0000, 200.0000,   0.9997,   1.0000,   4.0000],
        [351.0000, 197.0000,   0.9993,   1.0000,   5.0000],
        [371.0000, 206.0000,   0.9972,   1.0000,   6.0000]], device='cuda:0')
Vertices in Node features tensor([[258.0000, 367.0000,   1.0000,   1.0000,   1.0000],
        [258.0000, 283.0000,   0.9999,   1.0000,   2.0000],
        [233.0000, 207.0000,   0.9997,   1.0000,   3.0000],
        [252.0000, 200.0000,   0.9997,   1.0000,   4.0000],
        [351.0000, 197.0000,   0.9993,   1.0000,   5.0000],
        [371.0000, 206.0000,   0.9972,   1.0000,   6.0000]], device='cuda:0')
ground truth keypoints torch.Size([6, 4])
vertices for graph node features tensor([[258.0000, 367.0000,   1.0000,   1.0000,   1.0000],
        [258.0000, 283.0000,   0.9999,   1.0000,  

vertices for graph node features tensor([[258.0000, 367.0000,   1.0000,   1.0000,   1.0000],
        [258.0000, 283.0000,   0.9999,   1.0000,   2.0000],
        [281.0000, 207.0000,   0.9998,   1.0000,   3.0000],
        [301.0000, 213.0000,   0.9999,   1.0000,   4.0000],
        [323.0000, 309.0000,   0.9984,   1.0000,   5.0000],
        [339.0000, 297.0000,   0.9976,   1.0000,   6.0000]], device='cuda:0')
Vertices in Node features tensor([[258.0000, 367.0000,   1.0000,   1.0000,   1.0000],
        [258.0000, 283.0000,   0.9999,   1.0000,   2.0000],
        [281.0000, 207.0000,   0.9998,   1.0000,   3.0000],
        [301.0000, 213.0000,   0.9999,   1.0000,   4.0000],
        [323.0000, 309.0000,   0.9984,   1.0000,   5.0000],
        [339.0000, 297.0000,   0.9976,   1.0000,   6.0000]], device='cuda:0')
ground truth keypoints torch.Size([6, 4])
vertices for graph node features tensor([[258.0000, 367.0000,   1.0000,   1.0000,   1.0000],
        [258.0000, 283.0000,   0.9999,   1.0000,  

...

vertices for graph node features tensor([[258.0000, 367.0000,   1.0000,   1.0000,   1.0000],
        [258.0000, 283.0000,   0.9999,   1.0000,   2.0000],
        [282.0000, 206.0000,   0.9998,   1.0000,   3.0000],
        [301.0000, 213.0000,   0.9999,   1.0000,   4.0000],
        [400.0000, 215.0000,   0.9990,   1.0000,   5.0000],
        [395.0000, 195.0000,   0.9990,   1.0000,   6.0000]], device='cuda:0')
Vertices in Node features tensor([[258.0000, 367.0000,   1.0000,   1.0000,   1.0000],
        [258.0000, 283.0000,   0.9999,   1.0000,   2.0000],
        [282.0000, 206.0000,   0.9998,   1.0000,   3.0000],
        [301.0000, 213.0000,   0.9999,   1.0000,   4.0000],
        [400.0000, 215.0000,   0.9990,   1.0000,   5.0000],
        [395.0000, 195.0000,   0.9990,   1.0000,   6.0000]], device='cuda:0')
vertices for graph node features tensor([[258.0000, 367.0000,   1.0000,   1.0000,   1.0000],
        [258.0000, 283.0000,   0.9999,   1.0000,   2.0000],
        [281.0000, 206.0000,   0

vertices for graph node features tensor([[258.0000, 367.0000,   1.0000,   1.0000,   1.0000],
        [258.0000, 283.0000,   0.9999,   1.0000,   2.0000],
        [283.0000, 207.0000,   0.9996,   1.0000,   3.0000],
        [302.0000, 214.0000,   0.9997,   1.0000,   4.0000],
        [355.0000, 130.0000,   0.9987,   1.0000,   5.0000],
        [345.0000, 109.0000,   0.9976,   1.0000,   6.0000]], device='cuda:0')
Vertices in Node features tensor([[258.0000, 367.0000,   1.0000,   1.0000,   1.0000],
        [258.0000, 283.0000,   0.9999,   1.0000,   2.0000],
        [283.0000, 207.0000,   0.9996,   1.0000,   3.0000],
        [302.0000, 214.0000,   0.9997,   1.0000,   4.0000],
        [355.0000, 130.0000,   0.9987,   1.0000,   5.0000],
        [345.0000, 109.0000,   0.9976,   1.0000,   6.0000]], device='cuda:0')
vertices for graph node features tensor([[258.0000, 367.0000,   1.0000,   1.0000,   1.0000],
        [258.0000, 283.0000,   0.9999,   1.0000,   2.0000],
        [283.0000, 207.0000,   0

vertices for graph node features tensor([[258.0000, 367.0000,   1.0000,   1.0000,   1.0000],
        [258.0000, 283.0000,   0.9999,   1.0000,   2.0000],
        [305.0000, 219.0000,   0.9996,   1.0000,   3.0000],
        [322.0000, 231.0000,   0.9999,   1.0000,   4.0000],
        [420.0000, 216.0000,   0.9980,   1.0000,   5.0000],
        [432.0000, 198.0000,   0.9966,   1.0000,   6.0000]], device='cuda:0')
Vertices in Node features tensor([[258.0000, 367.0000,   1.0000,   1.0000,   1.0000],
        [258.0000, 283.0000,   0.9999,   1.0000,   2.0000],
        [305.0000, 219.0000,   0.9996,   1.0000,   3.0000],
        [322.0000, 231.0000,   0.9999,   1.0000,   4.0000],
        [420.0000, 216.0000,   0.9980,   1.0000,   5.0000],
        [432.0000, 198.0000,   0.9966,   1.0000,   6.0000]], device='cuda:0')
vertices for graph node features tensor([[258.0000, 367.0000,   1.0000,   1.0000,   1.0000],
        [258.0000, 283.0000,   0.9999,   1.0000,   2.0000],
        [305.0000, 219.0000,   0

vertices for graph node features tensor([[258.0000, 367.0000,   1.0000,   1.0000,   1.0000],
        [258.0000, 283.0000,   0.9999,   1.0000,   2.0000],
        [296.0000, 213.0000,   0.9997,   1.0000,   3.0000],
        [314.0000, 223.0000,   0.9998,   1.0000,   4.0000],
        [343.0000, 318.0000,   0.9972,   1.0000,   5.0000],
        [333.0000, 334.0000,   0.9853,   1.0000,   6.0000]], device='cuda:0')
Vertices in Node features tensor([[258.0000, 367.0000,   1.0000,   1.0000,   1.0000],
        [258.0000, 283.0000,   0.9999,   1.0000,   2.0000],
        [296.0000, 213.0000,   0.9997,   1.0000,   3.0000],
        [314.0000, 223.0000,   0.9998,   1.0000,   4.0000],
        [343.0000, 318.0000,   0.9972,   1.0000,   5.0000],
        [333.0000, 334.0000,   0.9853,   1.0000,   6.0000]], device='cuda:0')
vertices for graph node features tensor([[258.0000, 367.0000,   1.0000,   1.0000,   1.0000],
        [258.0000, 283.0000,   0.9999,   1.0000,   2.0000],
        [296.0000, 213.0000,   0

vertices for graph node features tensor([[258.0000, 367.0000,   1.0000,   1.0000,   1.0000],
        [258.0000, 283.0000,   0.9999,   1.0000,   2.0000],
        [310.0000, 222.0000,   0.9991,   1.0000,   3.0000],
        [325.0000, 236.0000,   0.9998,   1.0000,   4.0000],
        [375.0000, 322.0000,   0.9994,   1.0000,   5.0000],
        [370.0000, 339.0000,   0.9574,   1.0000,   6.0000]], device='cuda:0')
Vertices in Node features tensor([[258.0000, 367.0000,   1.0000,   1.0000,   1.0000],
        [258.0000, 283.0000,   0.9999,   1.0000,   2.0000],
        [310.0000, 222.0000,   0.9991,   1.0000,   3.0000],
        [325.0000, 236.0000,   0.9998,   1.0000,   4.0000],
        [375.0000, 322.0000,   0.9994,   1.0000,   5.0000],
        [370.0000, 339.0000,   0.9574,   1.0000,   6.0000]], device='cuda:0')
vertices for graph node features tensor([[258.0000, 367.0000,   1.0000,   1.0000,   1.0000],
        [258.0000, 283.0000,   0.9999,   1.0000,   2.0000],
        [311.0000, 223.0000,   0

In [ ]:
num_epochs = 50
batch_size = 4

KEYPOINTS_FOLDER_TRAIN = train_test_split(root_dir) +"/train" #train_test_split(root_dir) +"/train"
KEYPOINTS_FOLDER_VAL = train_test_split(root_dir) +"/val"
KEYPOINTS_FOLDER_TEST = train_test_split(root_dir) +"/test"

dataset_train = ClassDataset(KEYPOINTS_FOLDER_TRAIN, transform=train_transform(), demo=False)
dataset_val = ClassDataset(KEYPOINTS_FOLDER_VAL, transform=None, demo=False)
dataset_test = ClassDataset(KEYPOINTS_FOLDER_TEST, transform=None, demo=False)

data_loader_train = DataLoader(dataset_train, batch_size=batch_size, shuffle=True, collate_fn=collate_fn, pin_memory=True)
data_loader_val = DataLoader(dataset_val, batch_size=1, shuffle=False, collate_fn=collate_fn)
data_loader_test = DataLoader(dataset_test, batch_size=1, shuffle=False, collate_fn=collate_fn)

# Initialize the GradScaler for mixed precision training
scaler = GradScaler()

top_5_models = []

model.train()

for epoch in range(num_epochs):  # for 50 epochs
    for batch_idx, batch in enumerate(data_loader_train):
        images, targets = batch  
        
        if len(images) != batch_size:
            continue
        print("The training is continued")
        
        # Move images to GPU
        images = torch.stack(images).cuda()
        # Move targets to GPU
        for target in targets:
            for key, val in target.items():
                target[key] = val.cuda()
        
        ground_truth_keypoints = [target['keypoints'] for target in targets]
        ground_truth_boxes = [target['boxes'] for target in targets]
        
        # Assuming you want all images to be of size [3, 640, 480]
#         desired_size = (640, 480)  

        # Resize all images to the desired size
#         resized_images = [F.resize(img, desired_size) for img in images]

        ground_truth_keypoints = torch.stack(ground_truth_keypoints).squeeze()[:, :, 0:2]
        print("ground_truth_keypoints", ground_truth_keypoints)
        ground_truth_boxes = torch.stack(ground_truth_boxes)[:, :, 0:2]
        
        print("ground truth keypoints shape", ground_truth_keypoints.shape)

#         Create a batched adjacency matrix with the same batch size
#         batch_adj_matrix = adj_matrix.repeat(batch_size, 1, 1)
        batch_adj_matrix = adj_matrix
        print(batch_adj_matrix.device)
        
        optimizer.zero_grad()
        
#         # Forward pass for training
#         output_train = model(images, adj_matrix=batch_adj_matrix, targets=targets, train=True)
#         print("Output keypoints shape", output_train.keys())
        
#         #Forward pass for loss
#         predicted_keypoints = model(images, adj_matrix=batch_adj_matrix, train=False)
        
        
#         print("predicted keypoints", predicted_keypoints.shape)
                
#         loss_keypoint = output_train['loss_keypoint']
        
#         # Compute loss and backpropagate
#         loss = custom_loss(predicted_keypoints, ground_truth_keypoints, 
#                            adj_matrix=batch_adj_matrix, loss_keypoint=loss_keypoint)
        
#         loss.backward()
#         optimizer.step()

        # Automatic mixed precision for forward pass
        with autocast():
            output_train = model(images, targets=targets, train=True)
#             print("Output keypoints shape", output_train.keys())
            
            predicted_keypoints = model(images, train=False)
         
#             print("predicted keypoints", predicted_keypoints)

            loss_keypoint = output_train['loss_keypoint']

            # Compute loss
            loss = custom_loss(predicted_keypoints, ground_truth_keypoints, loss_keypoint=loss_keypoint)
            
            print(loss.device)
        
        # Scale the loss and backpropagate
        scaler.scale(loss).backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
        scheduler.step()
        scaler.step(optimizer)
        scaler.update()

        
        # Check if the current model should be saved as a top model
        if len(top_5_models) < 5 or loss.item() < max(top_5_models, key=lambda x: x[0])[0]:
            # Save the model state and loss
            model_state = {
                'epoch': epoch,
                'complete_model': model,
#                 'model_state_dict': model.state_dict(),
                'optimizer_state_dict': optimizer.state_dict(),
                'loss': loss.item(),
            }
            top_5_models.append((loss.item(), model_state))

            # Sort the list based on loss (ascending order)
            top_5_models.sort(key=lambda x: x[0])

            # If there are more than 5 models, remove the one with the highest loss
            if len(top_5_models) > 5:
                top_5_models.pop()

        print(f"Epoch {epoch+1}/{num_epochs}, Batch {batch_idx + 1}/{len(data_loader_train)}, Loss: {loss.item()}")
        
    print(f"Epoch {epoch+1}/{num_epochs}, Loss: {loss.item()}")
    
# After all epochs, save the top 5 models to disk
for idx, (_, model_state) in enumerate(top_5_models):
    torch.save(model_state, f'/home/jc-merlab/Pictures/Data/trained_models/best_gnn_model_b{batch_size}_e{num_epochs}_{idx+1}.pth')

In [ ]:
def visualize_and_save(img, vertices, filename):
    print("type of image befor conversion",type(img))    
    print("type of vertices before conversion", type(vertices))
    print(img.shape)
    img = (img.permute(1,2,0).cpu().numpy() * 255).astype(np.uint8)
#     img = (img * 255).astype(np.uint8)  # Convert back from [0, 1] range to [0, 255]
    img = cv2.cvtColor(img, cv2.COLOR_RGB2BGR)
    vertices = vertices.cpu().numpy()

    print(f"Image shape before saving: {img.shape}")  # print the image shape
    print("type of vertices", type(vertices))
#     print("entered vertices", vertices)
#     print("entered image", img)

    # Convert grayscale to BGR if necessary
    if len(img.shape) == 2:
        img = cv2.cvtColor(img, cv2.COLOR_GRAY2BGR)
        
    for i in range(vertices.shape[0]):
        img = cv2.circle(img, (int(vertices[i, 0]), int(vertices[i, 1])), radius=2, color=(0, 0, 255), thickness=-1)
        
    result = cv2.imwrite(filename, img)
    print(f"Image saved at {filename}: {result}")  # print if save was successful

In [ ]:
# Test loop
model.eval()  # Ensure the model is in evaluation mode

predicted_keypoints = []  # List to store predicted keypoints

with torch.no_grad():  # No need to track gradients in testing
    for idx, batch in enumerate(data_loader_test):
        img_tuple, target_dict_tuple, img_files = batch
        imgs = [img.to(device) for img in img_tuple]  # Create list of images
        total_test_loss = 0
        for i in range(len(imgs)):
            img = imgs[i].unsqueeze(0)  # Unsqueeze to add batch dimension
            # Prepare ground truth vertices for the image
            keypoints = target_dict_tuple[i]['keypoints'].to(device)
            visibility = torch.ones((keypoints.shape[0], keypoints.shape[1], 1)).to(device)
            vertices_gt = torch.cat((keypoints, visibility), dim=2).unsqueeze(0)  # Unsqueeze to add batch dimension
            vertices_gt = vertices_gt.squeeze()
            outputs = model(img)  # Forward pass
            
            vertices_pred, edge_pred, edge_features = outputs[0]

            loss = model.loss_function(vertices_pred, vertices_gt, edge_pred, edge_features)  # Compute the loss
            
            # Visualize and save the prediction
            filename = f'/home/jc-merlab/Pictures/Data/occ_vis_data/image_{idx}_{i}.jpg'
            visualize_and_save(img.squeeze(), vertices_pred, filename)
            print(f"Image saved at {filename}")  # Print statement to confirm image save

            total_test_loss += loss.item()
            predicted_keypoints.extend(outputs)

print(f'Test Loss: {total_test_loss/len(data_loader_test)}')

# Print predicted keypoints
for i, keypoints in enumerate(predicted_keypoints):
    print(f'Predicted keypoints for test image {i+1}:')
    print(keypoints)